In [ ]:
!nvidia-smi

Wed Mar 31 19:39:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install -U gitpython
!pip3 install transformers
!pip3 install urllib3
!pip3 install gputil
!pip3 install tabulate

     |████████████████████████████████| 163kB 5.7MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 2.0MB 5.8MB/s 
     |████████████████████████████████| 3.2MB 12.3MB/s 
     |████████████████████████████████| 890kB 36.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=f5fee4338619e14fa73db8a2e70c56e985d212d08d52da8a9ac9b94a4a26e44d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=3dca930e963977df6503f5a0c1e79ffb3e0e7debf231d3c039c06d85362eb3b4
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import re
import os
data = pd.read_csv('drive/MyDrive/dialogues.tsv', sep='\t')
data = data.drop(9635)
data = data.reset_index(drop=True)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import cv2
cv2.setNumThreads(0)

import argparse
from google.colab import drive



if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print('GPU')
else:
  device = torch.device("cpu")

GPU


In [6]:

import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler

model = AutoModelWithLMHead.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2').to(device)
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')



/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1010: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [7]:
SPECIAL_TOKENS = {'bos_token' : "<bos>", "eos_token" :"<eos>", 'additional_special_tokens': ["<speaker1>", "<speaker2>", "<fos>"]}
tokenizer.add_special_tokens(SPECIAL_TOKENS)
model.resize_token_embeddings(len(tokenizer))

Embedding(50263, 1024)

In [8]:
def merge_replyes(buffer):
    history = []
    i = 0
    a = 0
    b = a
    while (i < len(buffer)):
        same = ''
        a = i
        b = a
        while (i < len(buffer) and b < len(buffer) and buffer[a][0] == buffer[b][0]):
            b += 1
        
        for k in range(a, b):
            same += buffer[k][1] + '. '
        history.append([buffer[a][0], same])
        i = b
    return history

In [9]:
import re
import random
def generate_example(row):
    parsed = re.findall(r'Пользователь [12].+?(?=</span>)', data['dialogue'][row].replace("<br />", " "))
    history = merge_replyes([[1 if sentenses[13] == '1' else 2, sentenses[16:]] for sentenses in parsed])
    # print(history)
    random_slice = random.randint(1, min(len(history) - 1, 18))   # MAKE MAX 20 SENTENSES
    
    if history[random_slice][0] == 1:
        persona = re.findall(r'.+?(?=</span>)', data['persona_2_profile'][row].replace("<br />", " "))[0][26:]
    else:
        persona = re.findall(r'.+?(?=</span>)', data['persona_1_profile'][row].replace("<br />", " "))[0][26:]
        
    reply = history[random_slice][1]
    history = [[i[1]] for i in history[:random_slice]]
    
    
    return history, [reply], persona.split()
    

In [10]:
generate_example(2241)

([['Хай. '],
  ['Доброго времени суток! Рад пообщаться! Что делаете ?. '],
  ['Лежу загораю, А Вы,???????. Аууу. Вы где. Вы тоже меня хотите бросить. '],
  ['Круто! Вы наверное в путешествии? А я готовлю лекцию . Я преподаю религиезнавство в семинарии. Завтра сложный материал, голова уже кругом. Вот зашёл в чат немного отдохнуть.. '],
  ['И так всегда. Я ведь блогер. Обожаю путешествовать!!!. А у бога я не верю. А в бога я не верю. '],
  ['Я здесь 😊 просто у меня много кошек, а вечером они любят внимание 😁. Пытаюсь писать, а они отвлекают 😁😁😁. '],
  ['Приколямба😹😹😹👍👍👍👍🤟✌️✌️🤟🤟. Хахаха смеююююсь. '],
  ['Я бы тоже хотел куда- то поехать. А где Вы сейчас отдыхаете?. ']],
 ['Обожаю кошек. Я на островах. На Мадагаскаре. А вы чем занимаетесь. Какая у вас группа крови. Какой у вас рост. Я готовлю конфеты. '],
 ['я',
  'преподаватель',
  'я',
  'работаю',
  'в',
  'семинарии',
  'я',
  'живу',
  'рядом',
  'с',
  'работой',
  'люблю',
  'кошек',
  'у',
  'меня',
  '3',
  'брата'])

In [11]:
from itertools import chain

persona = [["Я", "люблю", "играть", "в", "баскетбол", "."],
           ["Я", "студент", "из", "Москвы", "."]]

history = [["привет", "как", "Дела", "?"],
           ["Все", "хорошо", "спасибо", "."]]

reply = ["Рад", "это", "слышать"]

bos, eos, speaker1, speaker2, fos = "<bos>", "<eos>", "<speaker1>", "<speaker2>", "<fos>"

def build_inputs(persona, history, reply):
    # Build our sequence by adding delimiters and concatenating
    sequence = [[bos] + list(chain(*persona))] + history + [[fos] + reply + [eos]]
    sequence = [sequence[0]] + [ [speaker2 if (len(sequence)-i) % 2 else speaker1] + s
                                for i, s in enumerate(sequence[1:])]
    # Build our word, segments and position inputs from the sequence
    words = list(chain(*sequence))                          # word tokens
    segments = [speaker2 if i % 2 else speaker1             # segment tokens
                for i, s in enumerate(sequence) for _ in s]
    position = list(range(len(words)))                      # position tokens
    return words, segments, position, sequence

words, segments, position, sequence = build_inputs(persona, history, reply)
print(words)

['<bos>', 'Я', 'люблю', 'играть', 'в', 'баскетбол', '.', 'Я', 'студент', 'из', 'Москвы', '.', '<speaker1>', 'привет', 'как', 'Дела', '?', '<speaker2>', 'Все', 'хорошо', 'спасибо', '.', '<speaker1>', '<fos>', 'Рад', 'это', 'слышать', '<eos>']


In [12]:
def get_example(index):
  history , reply, persona = generate_example(index)
  words, segments, position, sequence = build_inputs([persona], history, reply)
  # print(words)
  words = tokenizer(words)['input_ids']
  segments = tokenizer(segments)['input_ids']

  not_ansewer_tokens_len = len(list(chain(*words[:-len(sequence[-1])])))
  answer_tokens = list(chain(*words[-len(sequence[-1][1:]):]))
  words = list(chain(*words))
 
  # Prepare our language modeling targets: keep only the reply segment, -1 on the rest
  lm_targets = ([-100] * not_ansewer_tokens_len \
              + [-100] + answer_tokens)


  # Store the position of the last tokens for the next-sentence prediction loss
  last_token = len(words) - 1

  input_ids = torch.tensor(words, dtype=torch.long)

  token_type_ids = torch.tensor(segments, dtype=torch.long)


  mc_token_ids = torch.tensor(last_token, dtype=torch.long)


  lm_labels = torch.tensor(lm_targets, dtype=torch.long)
  # Next-sentence prediction labels
  mc_labels = torch.tensor([0], dtype=torch.long)  

  return input_ids, lm_labels

In [13]:
def prepare_input(persona, history):
    sequence = [[bos] + list(chain(*persona))] + history
    sequence = [sequence[0]] + [ [speaker2 if (len(sequence)-i) % 2 else speaker1] + s
                                for i, s in enumerate(sequence[1:])]
    # Build our word, segments and position inputs from the sequence
    words = list(chain(*sequence))                          # word tokens
    segments = [speaker2 if i % 2 else speaker1             # segment tokens
                for i, s in enumerate(sequence) for _ in s]
    position = list(range(len(words)))                      # position tokens            # position tokens

    print(words)
    words = tokenizer(words)['input_ids']
    words = list(chain(*words))
    input_ids = torch.tensor([words], dtype=torch.long)
    return input_ids

In [14]:
class Dialoges(Dataset):

    def __init__(self, csv_file):
        data = pd.read_csv(csv_file, sep='\t')
        data = data.drop(9635)
        data = data.reset_index(drop=True)
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return get_example(idx)

In [15]:
dataset = Dialoges('drive/MyDrive/dialogues.tsv')

train_loader = DataLoader(dataset,
                          batch_size=1, 
                          shuffle=False)

In [16]:
num_epochs = 3
learning_rate = 0.000003
warmup_steps = 50
total_steps = len(train_loader) * num_epochs
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [17]:
import GPUtil
from tabulate import tabulate

def train(model, loader, optimizer, scheduler, last_n_losses=200, verbose=True):

    losses = []

    progress_bar = tqdm(total=len(loader.dataset), disable=not verbose, desc='Train', position=0, leave=True)

    model.train()

    cnt = 0
    for X, y in loader:
        X = X.to(device)
        y = y.to(device)
        
        outputs = model(input_ids = X, labels = y)
        loss = outputs[0]
        losses.append(loss.item())

        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))
        
        progress_bar.update()
       

        if cnt % 1000 == 0 and cnt > 1000:
          save_last_model_path = 'drive/MyDrive/last_model_state_dict.pth'
          save_last_optimizer_path = 'drive/MyDrive/last_optimizer_state_dict.pth'
          torch.save(model.state_dict(), save_last_model_path)
          torch.save(optimizer.state_dict(), save_last_optimizer_path)


        if (cnt % 1000 == 0):
          # GPU information

          print("="*40, "GPU Details", "="*40)
          gpus = GPUtil.getGPUs()
          list_gpus = []
          for gpu in gpus:
              # get the GPU id
              gpu_id = gpu.id
              # name of GPU
              gpu_name = gpu.name
              # get % percentage of GPU usage of that GPU
              gpu_load = f"{gpu.load*100}%"
              # get free memory in MB format
              gpu_free_memory = f"{gpu.memoryFree}MB"
              # get used memory
              gpu_used_memory = f"{gpu.memoryUsed}MB"
              # get total memory
              gpu_total_memory = f"{gpu.memoryTotal}MB"
              # get GPU temperature in Celsius
              gpu_temperature = f"{gpu.temperature} °C"
              gpu_uuid = gpu.uuid
              list_gpus.append((
                  gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
                  gpu_total_memory, gpu_temperature, gpu_uuid
              ))

          print(tabulate(list_gpus, headers=("id", "name", "load", "free memory", "used memory", "total memory",
                                   "temperature", "uuid")))
        cnt += 1

    progress_bar.close()
    
    return losses

In [18]:
model.load_state_dict(torch.load('drive/MyDrive/last_model_state_dict.pth'))
optimizer.load_state_dict(torch.load('drive/MyDrive/last_optimizer_state_dict.pth'))

In [ ]:
train_losses = []
train_perplexities = []

for n_epoch in range(1, num_epochs + 1):
    
    epoch_train_losses = train(model, train_loader, optimizer, scheduler)
    mean_train_loss = np.mean(epoch_train_losses)
    
    train_losses.extend(epoch_train_losses)
    train_perplexities.append(np.exp(mean_train_loss))
    
    
    message = f'Epoch: {n_epoch}\n'
    message += f'Train: loss - {mean_train_loss:.4f} | perplexity - {train_perplexities[-1]:.3f}\n'  
    print(message) 
    torch.save(model.state_dict(), save_last_model_path)
    torch.save(optimizer.state_dict(), save_last_optimizer_path)
    # 1160/10012

In [ ]:
def speak():
  
  about = '<bos> Меня Зовут Анна, мне 18 лет. любою котят.'

  history = []
  print('Начните диалог')

  while(True):
    message = input()
    i = 1
    history.append([' <speaker1> ' + message])
    input_text = about + ''.join(list(chain(*history))) + ' <fos>' # history[-4:]

    inds = tokenizer(input_text, return_tensors='pt')['input_ids'].to(device)
    text = model.generate(
          input_ids=inds,
          repetition_penalty=5.0,
          top_k=5, 
          top_p=0.95, 
          temperature=1,
          max_length= len(inds[0]) + 20,
          do_sample=True,
          no_repeat_ngram_size=10,
          pad_token_id = 50259,
          # length_penalty = 0.5
          )
      
    answer = tokenizer.decode(text[0])

    history.append([' <speaker2> ' + answer[len(input_text):]])
    # print(input_text)
    # print(answer)
    print(answer[len(input_text):])
    # print(history)

      
      # print(answer[len(input_text):])
      # input_text += answer + ' '


In [ ]:
speak()

In [ ]:
arr = [len(generate_example(i)[0]) for i in range(2000, 2420)]


In [ ]:
print(np.argmax(arr), max(arr))


In [ ]:
np.mean(arr)

In [ ]:
generate_example(115)[0]